In [ ]:
from __future__ import annotations

from example_models import get_linear_chain_2v, get_poolman2000
from example_models.shared import constant

from modelbase2 import Model, Simulator

m = get_linear_chain_2v()
# print(m.get_full_concs({"x": 2.0, "y": 1.0}), "\n")
# print(m.get_args({"x": 2.0, "y": 1.0}), "\n")
# print(m.get_fluxes({"x": 2.0, "y": 1.0}), "\n")
# print(m.get_right_hand_side({"x": 2.0, "y": 1.0}), "\n")

Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10)
Simulator(m, {"x": 2.0, "y": 1.0}).simulate_to_steady_state()

v = Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10).get_fluxes()
c, v = Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10).get_concs_and_fluxes()
c, v = Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10).get_full_concs_and_fluxes()

In [ ]:
from modelbase2 import plot


def moiety_1(x1: float, total: float) -> float:
    return total - x1


m = (
    Model()
    .add_variables({"ATP": 1.0})
    .add_parameters({"ATP_total": 1.0})
    .add_derived("ADP", moiety_1, ["ATP", "ATP_total"])
    .add_reaction("ATPase", constant, {"ATP": -1}, ["ATP"])
)

c, v = Simulator(m).simulate(10).get_full_concs_and_fluxes()
if c is not None:
    plot.line(c)

In [ ]:
s = Simulator(get_poolman2000())